In [1]:
# native
import os
import sys
import time
import itertools

In [2]:
# external
import numpy as np
from matplotlib import pyplot as plt
import xmltodict

In [3]:
# custom
sys.path.append("../")

os.environ.update({
    "MAHNOB_DIR" : "",
    "EXPORT_DIR_BASE": ""
})

import emosm
from emosm.dataset.mahnob import mahnob
from emosm.dataset.mahnob import config
from emosm.sm import gazesm, physiosm

from emosm.plot import gazeplot

import emosm.fe.feature_extractor as fe

from emosm.tools import export

In [4]:
NOW = time.strftime("%y%m%d%H%M")

In [5]:
dataset = mahnob.Mahnob()

# sessions = dataset.get_session_by_id(10)
# sessions = dataset.get_session_by_id([10,160])
sessions = dataset.get_sessions_by_mediafile("53.avi")

Initializing MAHNOB dataset...


In [6]:
limit_frame = 500

In [7]:
def get_media_info(media, limit_frame=None):
    scale_media = config.SCALE_MEDIA
    display_size = media.get_size(scaled=scale_media)
    media_fps = media.metadata["fps"]
    media_frames_gen = media.get_frames(limit_frame=limit_frame, scale=scale_media, bw=True)
    return media_frames_gen, media_fps, display_size

In [10]:
def export_gaze_scanpath(sessions, limit_frame, destination):

    ## EXPORT GAZE SCANPATH FOR EACH SUBJECT

    for sid, session in sessions.items():

        media = session.get_media()
        media_frames_gen, media_fps, display_size = get_media_info(media=media, limit_frame=limit_frame)

        gaze_data = mahnob.Mahnob.collect_gaze_data(sessions={sid:session}, mapped=True)
        scanpath_generator = gazeplot.gaze_scanpath_plot_generator(gaze_data=gaze_data, limit_frame=limit_frame, fps=media_fps, display_size=display_size)

        if destination in ["video"]:

            filename = config.VIDEO_EXPORT_DIR_BASE + '/scanpath_{}_{}_{}.mp4'.format(sid, media.get_name(), NOW)
            export.toVideoSimple(data_frame_gen=scanpath_generator, media_frames_gen=media_frames_gen, filename=filename, fps=media_fps)

        elif destination in ["return"]:

            yield scanpath_generator

In [11]:
def export_gaze_sm(sessions, limit_frame, per_subject, destination):
    ## COMPUTE GAZE SALIENCY MAP FOR EACH SUBJECT

    if per_subject is False:

        for sid, session in sessions.items():
            media = session.get_media()
            break

        media_frames_gen, media_fps, display_size = get_media_info(media=media, limit_frame=limit_frame)

        gaze_data = mahnob.Mahnob.collect_gaze_data(sessions=sessions, mapped=True, preprocess=True)
        gaze_sm_gen = compute_gaze_sm(gaze_data=gaze_data, display_size=display_size, limit_frame=limit_frame)

        if destination in ["video"]:

            filename = config.VIDEO_EXPORT_DIR_BASE + '/gazesm_{}_{}.mp4'.format(media.get_name(), NOW)
            export.toVideo(sm_frame_gen=gaze_sm_gen, media_frames_gen=media_frames_gen, filename=filename, fps=media_fps)

        elif destination in ["return"]:

            return gaze_sm_gen

    else:

        for sid, session in sessions.items():

            media = session.get_media()
            media_frames_gen, media_fps, display_size = get_media_info(media=media, limit_frame=limit_frame)

            gaze_data = mahnob.Mahnob.collect_gaze_data(sessions={sid:session}, mapped=True)
            gaze_sm_gen = compute_gaze_sm(gaze_data=gaze_data, display_size=display_size, limit_frame=limit_frame)

            if destination in ["video"]:

                filename = config.VIDEO_EXPORT_DIR_BASE + '/gazesm_{}_{}_{}.mp4'.format(sid, media.get_name(), NOW)
                export.toVideo(sm_frame_gen=gaze_sm_gen, media_frames_gen=media_frames_gen, filename=filename, fps=media_fps)

            elif destination in ["return"]:

                return gaze_sm_gen

In [12]:
def show_physiological_signals_in_sessions(sessions, signals):

    ## LOAD AND SHOW PHYSIOLOGICAL DATA FOR GIVEN SESSSION

    for sid, session in sessions.items():
        physio_data = session.get_physiological_data(signals=signals)
        for sigtype, data in physio_data.items():
            data.get_data(preprocess=True, show=True)

In [13]:
def export_separated_physiological_saliency_map(sessions, media, signals, psyco_construct_list, attribute_list, limit_frame, destination):

    ## COMPUTE AND EXPORT SEPARATED PSYCOPHYSIOLOGICAL SALIENCY MAP FOR GIVEN SESSSION AND GIVEN SIGNALS

    media_frames_gen, media_fps, display_size = get_media_info(media=media, limit_frame=limit_frame)

    gaze_data = mahnob.Mahnob.collect_gaze_data(sessions=sessions, mapped=True)
    physio_data = mahnob.Mahnob.collect_physiological_data(sessions=sessions, signals=signals)
    for sigtype, data in physio_data.items():
        for psyco_construct in psyco_construct_list:
            for attribute in attribute_list:
                opts = {
                    "sigtype" : sigtype,
                    "attribute" : attribute,
                    "psyco_construct" : psyco_construct,
                    "fps" : media_fps
                }

                features = fe.extract_physiological_feature(data=data, opts=opts)
                psm = physiosm.PhysioSaliencyMap(physio=features, gaze=gaze_data, media=media)
                physio_sm_gen = psm.compute_saliency_map(limit_frame=limit_frame, display_size=display_size)

                if destination in ["video"]:

                    filename = config.VIDEO_EXPORT_DIR_BASE + '/physm_{}_{}_{}_{}.mp4'.format(psyco_construct, attribute, sigtype, NOW)
                    export.toVideo(sm_frame_gen=physio_sm_gen, media_frames_gen=media_frames_gen, filename=filename, fps=media_fps)

                elif destination in ["return"]:

                    yield physio_sm_gen

In [10]:
def export_composed_physiological_saliency_map(sessions, media, signals, psyco_construct, attribute, limit_frame, destination):

    ## COMPUTE AND EXPORT INTEGRATED PSYCOPHYSIOLOGICAL SALIENCY MAP FOR GIVEN SESSSION AND GIVEN SIGNALS

    media_frames_gen, media_fps, display_size = get_media_info(media=media, limit_frame=limit_frame)

    physio_sm_list = []

    gaze_data = mahnob.Mahnob.collect_gaze_data(sessions=sessions, mapped=True)
    physio_data = mahnob.Mahnob.collect_physiological_data(sessions=sessions, signals=signals)

    max_sample = min([ d.stop - d.start for k,data in physio_data.items() for d in data ])

    for sigtype, data in physio_data.items():
        opts = {
            "sigtype" : sigtype,
            "attribute" : attribute,
            "psyco_construct" : psyco_construct,
            "fps" : media_fps,
            "max_sample" : max_sample,
        }

        features = fe.extract_physiological_feature(data=data, opts=opts)
        psm = physiosm.PhysioSaliencyMap(physio=features, gaze=gaze_data, media=media)
        physio_sm_gen = psm.compute_saliency_map(limit_frame=limit_frame, display_size=display_size)
        physio_sm_list.append(physio_sm_gen)

    # create generator to integrate framed physiological data
    composed_sm = physiosm.physioSaliencyMapComposer(physio_sm_list)

    if destination in ["video"]:

        signals = "_".join(signals)
        filename = config.VIDEO_EXPORT_DIR_BASE + '/physm_composed_{}_{}_{}_{}.mp4'.format(psyco_construct, attribute, signals, NOW)
        export.toVideo(sm_frame_gen=composed_sm, media_frames_gen=media_frames_gen, filename=filename, fps=media_fps)

    elif destination in ["return"]:

        return composed_sm

In [15]:
def export_session_data_to_file(sessions, limit_frame):

    print "export_session_data_to_file"

    for sid, session in sessions.items():

        session_info = session.get_session_info()

        gaze_data = session.get_gaze_data()

        coordinates_raw = gaze_data.get_gaze_coordinates(mapped=True)
        coordinates = gaze_data.get_gaze_coordinates(mapped=True, preprocess=True)

        fixations_raw = gaze_data.get_fixations_data()
        fixations = gaze_data.get_fixations_data(preprocess=True)

        data = {
            "session_info" : session_info,
            "coordinates_raw" : coordinates_raw,
            "coordinates" : coordinates,
            "fixations_raw" : fixations_raw,
            "fixations" : fixations,
        }

        filename = config.DATA_EXPORT_DIR_BASE + "/gaze_data_{}_{}.npz".format(sid, NOW)
        export.toBinaryFile(data=data, filename=filename, compressed=True)

        del gaze_data
        del data

        physio_data = session.get_physiological_data(signals=["ECG","EDA","Resp","SKT"])

        if physio_data:
            signals = {}
            for signal, signal_class in physio_data.items():
                signals[signal.lower() + "_raw"] = signal_class.get_data()
                signals[signal.lower()] = signal_class.get_data(preprocess=True)

        data = dict(
            session_info=session_info,
            **signals
        )

        filename = config.DATA_EXPORT_DIR_BASE + "/physio_data_{}_{}.npz".format(sid, NOW)
        export.toBinaryFile(data=data, filename=filename, compressed=True)

        del session_info
        del physio_data
        del signals
        del data

In [ ]:
##
## EXPORT TO BINARY FILE
##

export_session_data_to_file(sessions=sessions, limit_frame=limit_frame)

In [ ]:
##
## EXPORT GAZE SCANPATH FOR EACH SUBJECT
##

export_gaze_scanpath(sessions=sessions, limit_frame=limit_frame)

In [ ]:
##
## COMPUTE GAZE SALIENCY MAP FOR EACH SUBJECT
##

export_gaze_sm(sessions=sessions, limit_frame=limit_frame, per_subject=True, destination="video")

In [ ]:
##
## COMPUTE GAZE SALIENCY MAP USING ALL SUBJECTS
##

export_gaze_sm(sessions=sessions, limit_frame=limit_frame, per_subject=False, destination="video")

# sm = export_gaze_sm(sessions=sessions, limit_frame=limit_frame, per_subject=False, destination="return")
# filename = config.DATA_EXPORT_DIR_BASE + "/gaze_sm_{}.npz".format(NOW)
# data = {
#     "gaze_sm" : sm
# }
# export.toBinaryFile(data=data, filename=filename, compressed=True)

In [ ]:
##
## LOAD AND SHOW PHYSIOLOGICAL DATA FOR GIVEN SESSSION
##

signals=["SKT"]
show_physiological_signals_in_sessions(sessions=sessions, signals=signals)

In [8]:
##
## EXPORT PHYSIOLOGICAL SALIENCY MAP
##

for sid, session in sessions.items():
    media = session.get_media()
    break

In [ ]:
##
## COMPUTE AND EXPORT SEPARATED PSYCOPHYSIOLOGICAL SALIENCY MAP FOR GIVEN SESSSION AND GIVEN SIGNALS
##

signals=["EDA"]
psyco_construct=["valence"]
attribute=["mean"]
export_separated_physiological_saliency_map(sessions=sessions, \
                                            media=media, \
                                            signals=signals, \
                                            psyco_construct_list=psyco_construct, \
                                            attribute_list=attribute, \
                                            limit_frame=limit_frame)

In [11]:
##
## COMPUTE AND EXPORT INTEGRATED PSYCOPHYSIOLOGICAL SALIENCY MAP FOR GIVEN SESSSION AND GIVEN SIGNALS
##

all_signals = ("ECG", "EDA", "Resp", "SKT")
all_psyco_construct = ["arousal", "valence"]
all_attribute = ["mean", "std"]

signal_combination = []
for s in map(set,itertools.product(all_signals, repeat=4)):
    if s not in signal_combination:
        signal_combination.append(s)

print sorted(signal_combination, key=len)

for signals, psyco_construct, attribute in itertools.product(signal_combination, all_psyco_construct, all_attribute):

    sm = export_composed_physiological_saliency_map(sessions=sessions, \
                                                    media=media, \
                                                    signals=signals, \
                                                    psyco_construct=psyco_construct, \
                                                    attribute=attribute, \
                                                    limit_frame=limit_frame,
                                                    destination="video")

    #data = { "physm" : list(sm) }
    #signals = "_".join(signals)
    #filename = config.DATA_EXPORT_DIR_BASE + "/physm_composed_{}_{}_{}_{}.npz".format(psyco_construct, attribute, signals, NOW)
    #export.toBinaryFile(data=data, filename=filename, compressed=True)
    break

[set(['ECG']), set(['EDA']), set(['Resp']), set(['SKT']), set(['ECG', 'EDA']), set(['ECG', 'Resp']), set(['ECG', 'SKT']), set(['Resp', 'EDA']), set(['SKT', 'EDA']), set(['SKT', 'Resp']), set(['ECG', 'Resp', 'EDA']), set(['ECG', 'SKT', 'EDA']), set(['ECG', 'SKT', 'Resp']), set(['SKT', 'Resp', 'EDA']), set(['ECG', 'SKT', 'Resp', 'EDA'])]
Load gaze data for sessions #10


/opt/conda/envs/python2/lib/python2.7/site-packages/scipy/signal/signaltools.py:2223: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/opt/conda/envs/python2/lib/python2.7/site-packages/scipy/signal/signaltools.py:2225: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/opt/conda/envs/python2/lib/python2.7/site-packages/scipy/signal/signaltools.py:2230: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpr

Load gaze data for sessions #160
Load gaze data for sessions #426
Load gaze data for sessions #546
Load gaze data for sessions #814
Load gaze data for sessions #948
Load gaze data for sessions #1206
Load gaze data for sessions #1588
Load gaze data for sessions #1698
Load gaze data for sessions #1956
Load gaze data for sessions #2090
Load gaze data for sessions #2224
Load gaze data for sessions #2358
Load gaze data for sessions #2492
Load gaze data for sessions #2626
Load gaze data for sessions #2760
Load gaze data for sessions #2894
Load gaze data for sessions #3028
Load gaze data for sessions #3412
Load gaze data for sessions #3538
Load gaze data for sessions #3664
Load gaze data for sessions #3790
Load physiological data for sessions #10
Load data from /home/jovyan/Dataset/Mahnob/data/Sessions/10/Part_1_S_Trial5_emotion.bdf..
Load ECG data..
Load physiological data for sessions #160
Load data from /home/jovyan/Dataset/Mahnob/data/Sessions/160/Part_2_S_Trial15_emotion.bdf..
Load ECG d

/opt/conda/envs/python2/lib/python2.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/opt/conda/envs/python2/lib/python2.7/site-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/opt/conda/envs/python2/lib/python2.7/site-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be in

prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
prepocess data ...
start compute saliency map
Process frame
# 0/500
(22, 3)


ValueError: too many values to unpack